In [1]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 14559421304260155768, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 9214062756
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 11777703292857114863
 physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"]

In [3]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, partial, rand, space_eval
from sklearn.metrics import log_loss
import sys
import time
import math
import os
import pandas as pd
from keras.callbacks import EarlyStopping
import numpy as np
from glob import glob
import cv2
import skimage
from skimage.transform import resize
from keras.utils.np_utils import to_categorical
import keras
from keras import layers
from keras import models
from keras import optimizers
from keras.models import load_model
# import keras.callbacks as kcall
from keras.optimizers import Adam, RMSprop,SGD
from keras.models import Model
from keras.models import Sequential
from keras.layers import Flatten, Dense, Activation, Dropout, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
# from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from keras.applications.vgg19 import VGG19
from keras.regularizers import l2

import matplotlib.pyplot as plt
from keras.layers import Input, concatenate
from keras import optimizers, metrics, models
from keras.layers import Input, Flatten, Dense

%matplotlib inline

Using TensorFlow backend.


In [4]:
import keras
import tensorflow as tf

print("Keras Version", keras.__version__)
print("tensorflow Version", tf.__version__)
# print("dim_ordering:", K.image_dim_ordering())

Keras Version 2.2.4
tensorflow Version 1.9.0


In [5]:
batch_size = 32
img_height, img_width = 128, 128
input_shape = (img_height, img_width, 3)
epochs = 15

In [6]:
train_dir = 'data/train/'
test_dir = 'data/test'

In [7]:
random_seed = np.random.seed(1142)

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    featurewise_center=True,
    featurewise_std_normalization=True,
    validation_split= 0.3,
    zoom_range=0.2,
    shear_range=0.2)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    subset = 'training',
    class_mode='categorical')

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    subset = 'validation',
    class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1. / 255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    class_mode='categorical')

Found 34909 images belonging to 6 classes.
Found 14958 images belonging to 6 classes.
Found 13578 images belonging to 6 classes.


In [8]:
nb_train_samples = len(train_generator.filenames)
nb_validation_samples = len(validation_generator.filenames)
nb_test_samples = len(test_generator.filenames)

predict_size_train = int(math.ceil(nb_train_samples / batch_size))
predict_size_validation = int(math.ceil(nb_validation_samples / batch_size))
predict_size_test = int(math.ceil(nb_test_samples / batch_size))

num_classes = len(train_generator.class_indices)

print("nb_train_samples:", nb_train_samples)
print("nb_validation_samples:", nb_validation_samples)
print("nb_test_samples:", nb_test_samples)

print("\npredict_size_train:", predict_size_train)
print("predict_size_validation:", predict_size_validation)
print("predict_size_test:", predict_size_test)

print("\n num_classes:", num_classes)

nb_train_samples: 34909
nb_validation_samples: 14958
nb_test_samples: 13578

predict_size_train: 1091
predict_size_validation: 468
predict_size_test: 425

 num_classes: 6


In [9]:
# vgg19_weights ="../input/full-keras-pretrained-no-top/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5"
# inception_weights ="../input/full-keras-pretrained-no-top//inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"
# vgg16_weights ="../input/full-keras-pretrained-no-top/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5"
# denseNet201_weights ="../input/full-keras-pretrained-no-top/densenet201_weights_tf_dim_ordering_tf_kernels_notop.h5"
# denseNet121_weights ="../input/full-keras-pretrained-no-top/densenet121_weights_tf_dim_ordering_tf_kernels_notop.h5"
# resenet50_weights ="../input/full-keras-pretrained-no-top/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5"
# inception_resnet_v2_weights ="../input/full-keras-pretrained-no-top/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5"
# nasnet_weights ="../input/full-keras-pretrained-no-top/nasnet_large_no_top.h5"
# nasnet_mobile_weights ="../input/full-keras-pretrained-no-top/nasnet_mobile_no_top.h5"
# mobilenet_weights ="../input/full-keras-pretrained-no-top/mobilenet_1_0_224_tf_no_top.h5"

In [10]:
# from keras.applications.vgg19 import VGG19
# from keras.applications.inception_v3 import InceptionV3
# from keras.applications.vgg16 import VGG16
# from keras.applications.xception import Xception, preprocess_input
# from keras.applications import DenseNet201
from keras.applications import DenseNet121
# from keras.applications import ResNet50
# from keras.applications.inception_resnet_v2 import InceptionResNetV2
# from keras.applications import NASNetLarge, NASNetMobile
# from keras.applications import MobileNet

bottleneck_final_model=DenseNet121(weights="imagenet", include_top=False, pooling = "avg")

In [11]:
# os.mkdir("extracted_features")
extracted_features_dir = "extracted_features/"
model_name = "DenseNet121_descriptors"

In [12]:
bottleneck_features_train = bottleneck_final_model.predict_generator(train_generator, predict_size_train, max_q_size=1, pickle_safe=False)
np.save(extracted_features_dir+'bottleneck_features_train_'+model_name+'.npy', bottleneck_features_train)

In [13]:
bottleneck_features_validation = bottleneck_final_model.predict_generator(validation_generator, predict_size_validation)
np.save(extracted_features_dir+'bottleneck_features_validation_'+model_name+'.npy', bottleneck_features_validation)

In [14]:
bottleneck_features_test = bottleneck_final_model.predict_generator(test_generator, predict_size_test)
np.save(extracted_features_dir+'bottleneck_features_test_'+model_name+'.npy', bottleneck_features_test)

In [15]:

import tensorflow as tf
from keras.backend.tensorflow_backend import get_session
from keras.backend.tensorflow_backend import clear_session
from keras.backend.tensorflow_backend import set_session

def reset_keras_tf_session():
    """
    this function clears the gpu memory and set the 
    tf session to not use the whole gpu
    """
    sess = get_session()
    clear_session()
    sess.close()
    sess = get_session()

#     config = tf.ConfigProto()
#     config.gpu_options.allow_growth = True
#     set_session(tf.Session(config=config))

reset_keras_tf_session()

In [16]:
train_data = np.load(extracted_features_dir+'bottleneck_features_train_'+model_name+'.npy')
validation_data = np.load(extracted_features_dir+'bottleneck_features_validation_'+model_name+'.npy')
test_data = np.load(extracted_features_dir+'bottleneck_features_test_'+model_name+'.npy')

train_labels = train_generator.classes
train_labels = to_categorical(train_labels, num_classes=num_classes)

validation_labels = validation_generator.classes
validation_labels = to_categorical(validation_labels, num_classes=num_classes)

test_labels = test_generator.classes
test_labels = to_categorical(test_labels, num_classes=num_classes)

In [17]:
space = {
         'lr': hp.choice('lr',[0.1, 0.01, 0.001, 0.0001, 0.00001,0.000001, 0.0000001]),
#          'dropout': hp.choice('dropout', [0.4, 0.5, 0.6, 0.7]),
#          'batch_size': hp.choice('batch_size', [64]),
#          'epochs': hp.choice('epochs', [15, 20, 25, 30, 50]),
#          'optimizer': hp.choice('optimizer',['sgd','adam','rmsprop']),
#          'optimizer': hp.choice('optimizer',['rmsprop']),
#          'optimizer': hp.choice('optimizer',['adam']),
         'beta_1':hp.choice('beta_1',[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8, 0.9]),
         'beta_2':hp.choice('beta_2',[0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.999,0.99,0.995]),
#          'momentum':hp.choice('momentum',[0.3,0.5,0.7,0.9,1]),
#          'amsgrad':hp.choice('amsgrad',[False,True]),
#          'nesterov':hp.choice('nesterov',[False,True]),
#          'rho':hp.choice('rho',[0.4,0.5,0.6,0.7,0.8,0.9,1]),
        }

In [18]:
from sklearn.metrics import accuracy_score
def f_nn(params):   
    print ('Parameters testing: ', params)
  
    adam_opt=Adam(lr=params["lr"], beta_1=params["beta_1"], beta_2=params['beta_2'], epsilon=None, decay=0.0)
#     sgd=SGD(lr=params["lr"], momentum=params['momentum'], decay=0.0, nesterov=params['nesterov'])
#     rmsprop=RMSprop(lr=params["lr"], rho=params['rho'], epsilon=None, decay=0.0)

    model = Sequential()
    # model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.7))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(optimizer=adam_opt, loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(train_data, train_labels,
                        epochs=epochs,
                        batch_size=batch_size,
                        validation_data=(validation_data, validation_labels),
                        verbose= 2,
                        callbacks=get_callbacks(params))

    (eval_loss, eval_accuracy) = model.evaluate(validation_data, validation_labels, batch_size= batch_size, verbose=1)

    print("Validation Accuracy: {:.4f}%".format(eval_accuracy * 100))
    print("Validation Loss: {}".format(eval_loss))
    
    filename = test_generator.filenames
    truth = test_generator.classes
    label = test_generator.class_indices
    indexlabel = dict((value, key) for key, value in label.items())

    preds = model.predict(test_data)

    predictions = [i.argmax() for i in preds]
    y_true = [i.argmax() for i in test_labels]
#     cm = confusion_matrix(y_pred=predictions, y_true=y_true)

    print('Test Accuracy: {}'.format(accuracy_score(y_true=y_true, y_pred=predictions)))

    print("*_*" * 50)
#     best_epoch = np.argmax(history.history['val_acc'])
#     best_val_acc = np.max(history.history['val_acc'])
#     print('Epoch {} - val acc: {}'.format(best_epoch, best_val_acc))
    sys.stdout.flush() 
    
    return {'loss': eval_loss, 'status': STATUS_OK, 'model': model}

In [19]:
def get_callbacks(params):
    callbacks =[EarlyStopping(monitor='val_acc', patience=3, verbose=1)]
    return callbacks

In [20]:
trials = Trials()
best = fmin(f_nn, space, algo=tpe.suggest, max_evals = 300, trials=trials)
print(best)

Parameters testing:  {'beta_1': 0.2, 'beta_2': 0.6, 'lr': 0.0001}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 5s - loss: 0.8980 - acc: 0.6766 - val_loss: 1.0459 - val_acc: 0.5476
Epoch 2/15
 - 4s - loss: 0.6246 - acc: 0.7784 - val_loss: 0.9984 - val_acc: 0.6153
Epoch 3/15
 - 4s - loss: 0.5681 - acc: 0.8016 - val_loss: 0.9721 - val_acc: 0.6340
Epoch 4/15
 - 4s - loss: 0.5404 - acc: 0.8176 - val_loss: 0.9921 - val_acc: 0.6393
Epoch 5/15
 - 4s - loss: 0.5195 - acc: 0.8249 - val_loss: 1.0476 - val_acc: 0.6419
Epoch 6/15
 - 4s - loss: 0.5083 - acc: 0.8320 - val_loss: 1.0154 - val_acc: 0.6517
Epoch 7/15
 - 4s - loss: 0.5012 - acc: 0.8385 - val_loss: 1.0248 - val_acc: 0.6575
Epoch 8/15
 - 4s - loss: 0.4914 - acc: 0.8421 - val_loss: 1.0270 - val_acc: 0.6664
Epoch 9/15
 - 4s - loss: 0.4881 - acc: 0.8464 - val_loss: 1.0120 - val_acc: 0.6815
Epoch 10/15
 - 4s - loss: 0.4868 - acc: 0.8475 - val_loss: 1.0473 - val_acc: 0.6721
Epoch 11/15
 - 4s - loss: 0.4803 - acc: 0.8499 - val_

14958/14958 [==============================] - 1s 34us/step
Validation Accuracy: 67.9837%
Validation Loss: 1.1021641765620962
Test Accuracy: 0.8082191780821918
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'beta_1': 0.5, 'beta_2': 0.8, 'lr': 1e-05}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 4s - loss: 1.7608 - acc: 0.3665 - val_loss: 1.3956 - val_acc: 0.4257
Epoch 2/15
 - 4s - loss: 1.0908 - acc: 0.6092 - val_loss: 1.2607 - val_acc: 0.4578
Epoch 3/15
 - 4s - loss: 0.9209 - acc: 0.6767 - val_loss: 1.1825 - val_acc: 0.4866
Epoch 4/15
 - 4s - loss: 0.8372 - acc: 0.7022 - val_loss: 1.1369 - val_acc: 0.5047
Epoch 5/15
 - 4s - loss: 0.7848 - acc: 0.7217 - val_loss: 1.0960 - val_acc: 0.5202
Epoch 6/15
 - 4s - loss: 0.7451 - acc: 0.7325 - val_loss: 1.0790 - val_acc: 0.5263
Epoch 7/15
 - 4s - loss: 0.7169 - acc: 0.7410 - val_loss: 1.0578 - val_acc: 

Parameters testing:  {'beta_1': 0.5, 'beta_2': 0.8, 'lr': 0.001}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 5s - loss: 0.6344 - acc: 0.7783 - val_loss: 0.9496 - val_acc: 0.6712
Epoch 2/15
 - 4s - loss: 0.5239 - acc: 0.8263 - val_loss: 0.9965 - val_acc: 0.7008
Epoch 3/15
 - 4s - loss: 0.5163 - acc: 0.8386 - val_loss: 1.0542 - val_acc: 0.6826
Epoch 4/15
 - 4s - loss: 0.5277 - acc: 0.8425 - val_loss: 1.1798 - val_acc: 0.6779
Epoch 5/15
 - 4s - loss: 0.5225 - acc: 0.8474 - val_loss: 1.3121 - val_acc: 0.6617
Epoch 00005: early stopping
14958/14958 [==============================] - 1s 36us/step
Validation Accuracy: 66.1653%
Validation Loss: 1.3120789448257686
Test Accuracy: 0.8006333775224628
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'beta_1': 0.8, 'beta_2': 0.7, 'lr': 0.01}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 5s 

 - 4s - loss: 0.5494 - acc: 0.8075 - val_loss: 0.9776 - val_acc: 0.6356
Epoch 4/15
 - 4s - loss: 0.5241 - acc: 0.8218 - val_loss: 0.9707 - val_acc: 0.6459
Epoch 5/15
 - 4s - loss: 0.5050 - acc: 0.8303 - val_loss: 1.0033 - val_acc: 0.6437
Epoch 6/15
 - 4s - loss: 0.4917 - acc: 0.8359 - val_loss: 0.9680 - val_acc: 0.6637
Epoch 7/15
 - 4s - loss: 0.4846 - acc: 0.8411 - val_loss: 1.0051 - val_acc: 0.6604
Epoch 8/15
 - 4s - loss: 0.4768 - acc: 0.8439 - val_loss: 1.0126 - val_acc: 0.6657
Epoch 9/15
 - 4s - loss: 0.4688 - acc: 0.8500 - val_loss: 1.0715 - val_acc: 0.6669
Epoch 10/15
 - 4s - loss: 0.4688 - acc: 0.8512 - val_loss: 1.0804 - val_acc: 0.6719
Epoch 11/15
 - 4s - loss: 0.4674 - acc: 0.8546 - val_loss: 1.0656 - val_acc: 0.6749
Epoch 12/15
 - 4s - loss: 0.4658 - acc: 0.8577 - val_loss: 1.0569 - val_acc: 0.6816
Epoch 13/15
 - 4s - loss: 0.4643 - acc: 0.8563 - val_loss: 1.0873 - val_acc: 0.6847
Epoch 14/15
 - 4s - loss: 0.4678 - acc: 0.8607 - val_loss: 1.0830 - val_acc: 0.6812
Epoch 15/1

Epoch 6/15
 - 4s - loss: 0.4383 - acc: 0.8439 - val_loss: 0.8668 - val_acc: 0.6691
Epoch 7/15
 - 4s - loss: 0.4204 - acc: 0.8518 - val_loss: 0.8821 - val_acc: 0.6654
Epoch 8/15
 - 4s - loss: 0.4099 - acc: 0.8545 - val_loss: 0.8776 - val_acc: 0.6740
Epoch 9/15
 - 4s - loss: 0.3965 - acc: 0.8585 - val_loss: 0.8399 - val_acc: 0.6900
Epoch 10/15
 - 4s - loss: 0.3821 - acc: 0.8651 - val_loss: 0.8718 - val_acc: 0.6775
Epoch 11/15
 - 4s - loss: 0.3749 - acc: 0.8703 - val_loss: 0.8802 - val_acc: 0.6786
Epoch 12/15
 - 4s - loss: 0.3692 - acc: 0.8695 - val_loss: 0.8530 - val_acc: 0.6915
Epoch 13/15
 - 4s - loss: 0.3641 - acc: 0.8720 - val_loss: 0.8742 - val_acc: 0.6836
Epoch 14/15
 - 4s - loss: 0.3561 - acc: 0.8748 - val_loss: 0.8717 - val_acc: 0.6960
Epoch 15/15
 - 4s - loss: 0.3460 - acc: 0.8768 - val_loss: 0.8641 - val_acc: 0.6996
14958/14958 [==============================] - 1s 39us/step
Validation Accuracy: 69.9559%
Validation Loss: 0.8640905685155242
Test Accuracy: 0.8264840182648402
*_**

...

Epoch 5/15
 - 12s - loss: 0.4519 - acc: 0.8398 - val_loss: 0.8854 - val_acc: 0.6474
Epoch 6/15
 - 11s - loss: 0.4340 - acc: 0.8455 - val_loss: 0.8582 - val_acc: 0.6646
Epoch 7/15
 - 12s - loss: 0.4158 - acc: 0.8512 - val_loss: 0.8777 - val_acc: 0.6653
Epoch 8/15
 - 10s - loss: 0.4051 - acc: 0.8577 - val_loss: 0.9052 - val_acc: 0.6673
Epoch 9/15
 - 11s - loss: 0.3945 - acc: 0.8604 - val_loss: 0.8826 - val_acc: 0.6752
Epoch 10/15
 - 11s - loss: 0.3840 - acc: 0.8658 - val_loss: 0.8061 - val_acc: 0.7008
Epoch 11/15
 - 12s - loss: 0.3741 - acc: 0.8676 - val_loss: 0.9269 - val_acc: 0.6791
Epoch 12/15
 - 12s - loss: 0.3646 - acc: 0.8724 - val_loss: 0.8924 - val_acc: 0.6819
Epoch 13/15
 - 11s - loss: 0.3589 - acc: 0.8729 - val_loss: 0.8887 - val_acc: 0.6852
Epoch 00013: early stopping
14958/14958 [==============================] - 2s 135us/step
Validation Accuracy: 68.5185%
Validation Loss: 0.8886892422201887
Test Accuracy: 0.8214759169244366
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*

Epoch 12/15
 - 11s - loss: 0.4657 - acc: 0.8632 - val_loss: 1.1544 - val_acc: 0.6830
Epoch 13/15
 - 10s - loss: 0.4713 - acc: 0.8644 - val_loss: 1.2135 - val_acc: 0.6852
Epoch 14/15
 - 11s - loss: 0.4718 - acc: 0.8654 - val_loss: 1.1912 - val_acc: 0.6933
Epoch 15/15
 - 11s - loss: 0.4783 - acc: 0.8652 - val_loss: 1.2428 - val_acc: 0.6911
14958/14958 [==============================] - 2s 148us/step
Validation Accuracy: 69.1068%
Validation Loss: 1.242827123522139
Test Accuracy: 0.8168360583296509
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'beta_1': 0.1, 'beta_2': 0.995, 'lr': 0.1}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 23s - loss: 12.4104 - acc: 0.2287 - val_loss: 12.4695 - val_acc: 0.2264
Epoch 2/15
 - 11s - loss: 12.4203 - acc: 0.2292 - val_loss: 12.4695 - val_acc: 0.2264
Epoch 3/15
 - 11s - loss: 12.3600 - acc: 0.2327 - val_loss: 12

Epoch 7/15
 - 10s - loss: 0.4192 - acc: 0.8504 - val_loss: 0.8878 - val_acc: 0.6685
Epoch 8/15
 - 10s - loss: 0.4054 - acc: 0.8565 - val_loss: 0.8317 - val_acc: 0.6906
Epoch 9/15
 - 11s - loss: 0.3881 - acc: 0.8630 - val_loss: 0.8791 - val_acc: 0.6776
Epoch 10/15
 - 11s - loss: 0.3821 - acc: 0.8645 - val_loss: 0.8876 - val_acc: 0.6758
Epoch 11/15
 - 10s - loss: 0.3719 - acc: 0.8673 - val_loss: 0.9014 - val_acc: 0.6733
Epoch 00011: early stopping
14958/14958 [==============================] - 2s 162us/step
Validation Accuracy: 67.3285%
Validation Loss: 0.9013939449502107
Test Accuracy: 0.8102076889085285
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'beta_1': 0.9, 'beta_2': 0.99, 'lr': 0.0001}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 22s - loss: 0.8833 - acc: 0.6764 - val_loss: 0.9921 - val_acc: 0.5687
Epoch 2/15
 - 11s - loss: 0.5826 - ac

Parameters testing:  {'beta_1': 0.6, 'beta_2': 0.4, 'lr': 0.1}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 24s - loss: 12.1863 - acc: 0.2427 - val_loss: 12.4695 - val_acc: 0.2264
Epoch 2/15
 - 12s - loss: 12.4393 - acc: 0.2277 - val_loss: 12.4695 - val_acc: 0.2264
Epoch 3/15
 - 12s - loss: 12.4251 - acc: 0.2283 - val_loss: 12.4695 - val_acc: 0.2264
Epoch 4/15
 - 11s - loss: 12.2960 - acc: 0.2366 - val_loss: 12.4695 - val_acc: 0.2264
Epoch 00004: early stopping
14958/14958 [==============================] - 2s 138us/step
Validation Accuracy: 22.6367%
Validation Loss: 12.469487922557265
Test Accuracy: 0.2267638827515098
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'beta_1': 0.8, 'beta_2': 0.3, 'lr': 0.0001}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 22s - loss: 0.7594 - acc: 0.7280 - val_loss: 0.9756 - val_acc: 0.6155
Epo

 - 10s - loss: 0.6369 - acc: 0.7705 - val_loss: 0.9984 - val_acc: 0.5620
Epoch 12/15
 - 12s - loss: 0.6263 - acc: 0.7765 - val_loss: 0.9911 - val_acc: 0.5676
Epoch 13/15
 - 12s - loss: 0.6154 - acc: 0.7786 - val_loss: 0.9792 - val_acc: 0.5781
Epoch 14/15
 - 11s - loss: 0.6046 - acc: 0.7825 - val_loss: 0.9720 - val_acc: 0.5804
Epoch 15/15
 - 10s - loss: 0.5956 - acc: 0.7877 - val_loss: 0.9692 - val_acc: 0.5872
14958/14958 [==============================] - 2s 150us/step
Validation Accuracy: 58.7244%
Validation Loss: 0.9692135808942335
Test Accuracy: 0.742966563558698
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
{'beta_1': 0, 'lr': 3, 'beta_2': 9}
